# iTensors.jl

This notebook serves as a test run of the example code given in the documentation at https://docs.itensor.org/ITensors/stable/#Singular-Value-Decomposition-(SVD)-of-a-Tensor

This is purely a copy paste of the iTensor code samples, testing the functionality inside a `.ipynb` environment.

**References**

1. Matthew Fishman, Steven R. White, and E. Miles Stoudenmire, *The ITensor Software Library for Tensor Network Calculations*, SciPost Phys. Codebases, 4 (2022), SciPost. [doi:10.21468/SciPostPhysCodeb.4](https://doi.org/10.21468/SciPostPhysCodeb.4).

2. Matthew Fishman, Steven R. White, and E. Miles Stoudenmire, *Codebase release 0.3 for ITensor*, SciPost Phys. Codebases, 4-r0.3 (2022), SciPost. [doi:10.21468/SciPostPhysCodeb.4-r0.3](https://doi.org/10.21468/SciPostPhysCodeb.4-r0.3).


# Basic Overview
ITensor construction, setting of elements, contraction, and addition. Before constructing an ITensor, one constructs Index objects representing tensor indices.

In [ ]:
using ITensors

In [ ]:
let
  i = Index(3)
  j = Index(5)
  k = Index(2)
  l = Index(7)

  A = ITensor(i,j,k)
  B = ITensor(j,l)

  # Set elements of A
  A[i=>1,j=>1,k=>1] = 11.1
  A[i=>2,j=>1,k=>2] = -21.2
  A[k=>1,i=>3,j=>1] = 31.1  # can provide Index values in any order
  # ...

  # Contract over shared index j
  C = A * B

  @show hasinds(C,i,k,l) # = true

  D = random_itensor(k,j,i) # ITensor with random elements

  # Add two ITensors
  # must have same set of indices
  # but can be in any order
  R = A + D

  nothing
end

hasinds(C, i, k, l) = true


# Singular Value Decomposition (SVD) of a Matrix
In this example, we create a random 10x20 matrix and compute its SVD. The resulting factors can be simply multiplied back together using the ITensor * operation, which automatically recognizes the matching indices between U and S, and between S and V and contracts (sums over) them.

In [ ]:
let
  i = Index(10)           # index of dimension 10
  j = Index(20)           # index of dimension 20
  M = random_itensor(i,j)  # random matrix, indices i,j
  U,S,V = svd(M,i)        # compute SVD with i as row index
  @show M ≈ U*S*V         # = true

  nothing
end

M ≈ U * S * V = true


# Singular Value Decomposition (SVD) of a Tensor
In this example, we create a random 4x4x4x4 tensor and compute its SVD, temporarily treating the indices i and k together as the "row" index and j and l as the "column" index for the purposes of the SVD. The resulting factors can be simply multiplied back together using the ITensor * operation, which automatically recognizes the matching indices between U and S, and between S and V and contracts (sums over) them.

In [6]:
let
  i = Index(4,"i")
  j = Index(4,"j")
  k = Index(4,"k")
  l = Index(4,"l")
  T = random_itensor(i,j,k,l)
  U,S,V = svd(T,i,k)   # compute SVD with (i,k) as row indices (indices of U)
  @show hasinds(U,i,k) # = true
  @show hasinds(V,j,l) # = true
  @show T ≈ U*S*V      # = true

  nothing
end

hasinds(U, i, k) = true
hasinds(V, j, l) = true
T ≈ U * S * V = true


# Tensor Indices: Tags and Prime Levels
Before making an ITensor, you have to define its indices. Tensor Index objects carry extra information beyond just their dimension.

All Index objects carry a permanent, immutable id number which is determined when it is constructed, and allow it to be matched (compare equal) with copies of itself.

Additionally, an Index can have up to four tag strings, and an integer primelevel. If two Index objects have different tags or different prime levels, they do not compare equal even if they have the same id.

Tags are also useful for identifying Index objects when printing tensors, and for performing certain Index manipulations (e.g. priming indices having certain sets of tags).

In [7]:
let
  i = Index(3)     # Index of dimension 3
  @show dim(i)     # = 3
  @show id(i)      # = 0x5d28aa559dd13001 or similar

  ci = copy(i)
  @show ci == i    # = true

  j = Index(5,"j") # Index with a tag "j"

  @show j == i     # = false

  s = Index(2,"n=1,Site") # Index with two tags,
                          # "Site" and "n=1"
  @show hastags(s,"Site") # = true
  @show hastags(s,"n=1")  # = true

  i1 = prime(i) # i1 has a "prime level" of 1
                # but otherwise same properties as i
  @show i1 == i # = false, prime levels do not match

  nothing
end

dim(i) = 3
id(i) = 0x9d92fa5912cc01f0
ci == i = true
j == i = false
hastags(s, "Site") = true
hastags(s, "n=1") = true
i1 == i = false
